In [ ]:
import numpy as np
import csv
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt

In [ ]:
import camera_tools as ct

In [ ]:
from cmac2 import CMAC

In [ ]:
#Calibrate the camera to detect green box, if you haven't done this calibration before
low_green, high_green = ct.colorpicker()
print(low_green)
print(high_green)

In [ ]:
low_red, high_red = ct.colorpicker()
print(low_red)
print(high_red)

In [ ]:
#Check whether the camera detects the green object properly
cam = ct.prepare_camera()
image = ct.capture_image(cam)
x,y = ct.locate(image, low_green, high_green)
ct.show_camera(cam)

print(x,y)

In [ ]:
#Check whether the camera detects the green object properly
cam_red = ct.prepare_camera()
img_red = ct.capture_image(cam_red)
ct.show_camera(cam_red, low_red, high_red)

In [ ]:
from FableAPI.fable_init import api
api.setup(blocking=True)

In [ ]:
moduleids = api.discoverModules()
print("Module IDs: ", moduleids)
moduleID = moduleids[0]
print("Battery level:",api.getBattery(moduleID),"%")

In [ ]:
#Create two files (if they were not already created) to collect data.
if (not os.path.exists("xycoords_2.csv")):
    f = open('xycoords_2.csv', 'w')
    with f:
        writer = csv.writer(f)
        writer.writerows([["X","Y"]])
    f.close()
if (not os.path.exists("angles_2.csv")):
    f = open('angles_2.csv','w')
    with f:
        writer = csv.writer(f)
        writer.writerows([["Y_angle"]])
    f.close()

In [ ]:
n_t1 = 20
n_t2 = 20

t1 = np.tile(np.linspace(-85, 86, n_t1), n_t2) # repeat the vector
t2 = np.repeat(np.linspace(0, 86, n_t2), n_t1) # repeat each element
thetas = np.stack((t1,t2))

num_datapoints = n_t1*n_t2

# # Plotting
# plt.figure(figsize=(6, 6))
# plt.scatter(thetas[0], thetas[1])
# plt.xlabel('t1')
# plt.ylabel('t2')
# plt.title('Grid of Points')
# plt.grid(True)
# plt.show()

In [ ]:
ourdata = np.zeros((4, num_datapoints))

def collect_data_2D(num_datapoints):
    for i in range(num_datapoints):
        api.setPos(thetas[0,i], thetas[1,i], moduleID)
        img = ct.capture_image(cam)
        x, y = ct.locate(img, low_green, high_green)
        tmeas1 = api.getPos(0,moduleID)
        tmeas2 = api.getPos(1,moduleID)
        ourdata = np.array([tmeas1, tmeas2, x, y])
        with open('xycoords_2.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerows([ourdata[2:4]])
        with open('angles_2.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerows([ourdata[0:2]])
        api.sleep(2)
    return None
  

In [ ]:
collect_data_2D(num_datapoints)

In [ ]:
#we use the collectData function to collect the data for training.
    #we collect Y (angular) position of the end effector
    #we collect x,y coordinates of the end effector in the camera image
    #see the video final_project_guidance.mp4
def collectData(desired_angle_change):
    cam = ct.prepare_camera()
    ct.show_camera(cam)
    cam.release()
    cam = ct.prepare_camera()
    api.setPos(-90,90,moduleID)
    api.sleep(1.5)
    Y_angle_list = []
    XY_coordinates_list = []
    traversed_directions = 0
    current_direction = 0     # 1 is for clockwise, 0 is for anticlockwise
    traversedDirections = 0
    num_of_iterations = (int)(np.round(360/desired_angle_change,1))
    #Fable's Y arm traverse anticlockwise
    if current_direction == 0:
        for i in range(num_of_iterations):
            img = ct.capture_image(cam)
            x,y = ct.locate(img, low_green, high_green)
            currentRobotYAng = (int)(np.round(api.getPos(1,moduleID),1))
            Y_angle_list.append([currentRobotYAng])
            XY_coordinates_list.append([x,y])
            currentRobotYAng = currentRobotYAng - (desired_angle_change)
            api.setPos(-90,currentRobotYAng,moduleID)
            api.sleep(1.5)
            if np.abs(currentRobotYAng) > 90:
                current_direction = 1
                traversedDirections = traversedDirections + 1
                break
    #Fable's Y Arm traverses clockwise
    if current_direction == 1:
        for i in range(num_of_iterations):
            img = ct.capture_image(cam)
            x,y = ct.locate(img, low_green, high_green)
            currentRobotYAng = (int)(np.round(api.getPos(1,moduleID),1))
            Y_angle_list.append([currentRobotYAng])
            XY_coordinates_list.append([x,y])
            currentRobotYAng = currentRobotYAng + (desired_angle_change)
            api.setPos(-90,currentRobotYAng,moduleID)
            api.sleep(1.5)
            if np.abs(currentRobotYAng) > 90:
                current_direction = 0
                traversedDirections = traversedDirections + 1
                break
    if traversedDirections == 2:
        cam.release()
        #Save collected data to files
        y_angle_file_ptr = open('angles_2.csv', 'a+', newline ='')
        with y_angle_file_ptr:
            writer = csv.writer(y_angle_file_ptr)
            writer.writerows(Y_angle_list)
        y_angle_file_ptr.close()
        file_xycoords = open('xycoords_2.csv', 'a+', newline ='')
        with file_xycoords:
            writer = csv.writer(file_xycoords)
            writer.writerows(XY_coordinates_list)
        file_xycoords.close()
    return
    

In [ ]:
#TODO: Call the CollectData() function with different values for 'desired_angle_change' argument and collect sufficient
#sufficient amout of data to angles_2.csv file and xycoords_2.csv file.
collectData(5)

In [ ]:
#Use this function to calculate Y (angular) position errors 
# Y (angular) positions
def readAngleFilesAndCollectErrors(fileName):
    angleDataFrame = pd.read_csv(fileName)
    desired1_column = angleDataFrame.columns[0]
    desired2_column = angleDataFrame.columns[1]
    angle1_error_list = []
    angle2_error_list = []
    for i in range(1,len(angleDataFrame)):
        current1_data_element = angleDataFrame[desired1_column][i]
        previous1_data_element = angleDataFrame[desired1_column][i-1]
        current2_data_element = angleDataFrame[desired2_column][i]
        previous2_data_element = angleDataFrame[desired2_column][i-1]

        #TODO: Calculate the Y (angular) position error(say current_error) as the difference between 
        # the current Y (angular) position and previous Y (angular) position

        current_error1 = current1_data_element - previous1_data_element
        current_error2 = current2_data_element - previous2_data_element

        #print(current_data_element," ", previous_data_element, " ", current_error)
        angle1_error_list = angle1_error_list + [current_error1]
        angle2_error_list = angle2_error_list + [current_error2]

    return [angle1_error_list, angle2_error_list] 

In [ ]:
#Use this function to calculate errors between x and y coordinates
def readXYCoordsFilesAndCollectErrors(fileName):
    xyCoordsDataFrame = pd.read_csv(fileName)
    x_col_name = xyCoordsDataFrame.columns[0]
    y_col_name = xyCoordsDataFrame.columns[1]
    x_pos_error_list = []
    y_pos_error_list = []
    for i in range(1,len(xyCoordsDataFrame)):
        current_data_element_x = xyCoordsDataFrame[x_col_name][i]
        current_data_element_y = xyCoordsDataFrame[y_col_name][i]
        previous_data_element_x = xyCoordsDataFrame[x_col_name][i-1]
        previous_data_element_y = xyCoordsDataFrame[y_col_name][i-1]
        error_x_pos = current_data_element_x - previous_data_element_x 
        error_y_pos = current_data_element_y - previous_data_element_y
        x_pos_error_list = x_pos_error_list + [error_x_pos]
        y_pos_error_list = y_pos_error_list + [error_y_pos] 
    return [x_pos_error_list,y_pos_error_list]

In [ ]:
#Read the provided files and load the calculated errors into to appropriate lists as follows
# angle_error_list_1 = readAngleFilesAndCollectErrors("angles_1.csv")
# x_pos_error_list_1,y_pos_error_list_1 = readXYCoordsFilesAndCollectErrors("xyCoords_1.csv")
#Read the files with the data dumped by you and load the calculated errors into appropriate lists.
angle1_error_list_2, angle2_error_list_2 = readAngleFilesAndCollectErrors("angles_2.csv")
x_pos_error_list_2,y_pos_error_list_2 = readXYCoordsFilesAndCollectErrors("xyCoords_2.csv")

Remove first entry


In [ ]:
angle1_error_list_2 = angle1_error_list_2[1:]
angle2_error_list_2 = angle2_error_list_2[1:]
x_pos_error_list_2 = x_pos_error_list_2[1:]
y_pos_error_list_2 = y_pos_error_list_2[1:]

Standardize errors

In [ ]:
# mean_x_pos_1 = np.mean(x_pos_error_list_1)
# mean_y_pos_1 = np.mean(y_pos_error_list_1)
# std_x_pos_1 = np.std(x_pos_error_list_1)
# std_y_pos_1 = np.std(y_pos_error_list_1)
# x_pos_error_list_2 = np.array(x_pos_error_list_2[1:])
# mean_x_pos_2 = np.mean(x_pos_error_list_2)
# mean_y_pos_2 = np.mean(y_pos_error_list_2)
# std_x_pos_2 = np.std(x_pos_error_list_2)
# std_y_pos_2 = np.std(y_pos_error_list_2)
# print(mean_x_pos_2,mean_y_pos_2,std_x_pos_2,std_y_pos_2)

In [ ]:
# angle_error_list_1 = (angle_error_list_1 - np.mean(angle_error_list_1))/np.std(angle_error_list_1)
# x_pos_error_list_1 = (x_pos_error_list_1 - np.mean(x_pos_error_list_1,0))/np.std(x_pos_error_list_1,0)
# y_pos_error_list_1 = (y_pos_error_list_1 - np.mean(y_pos_error_list_1,0))/np.std(y_pos_error_list_1,0)
# # print("Angle error list 1: ",angle_error_list_1.shape)
# print("X position error list 1: ",x_pos_error_list_1.shape)
# print("Y position error list 1: ",y_pos_error_list_1.shape)
# angle_error_list_2 = (angle_error_list_2 - np.mean(angle_error_list_2))/np.std(angle_error_list_2)
#--------------------------------------
# x_pos_error_list_2 = (x_pos_error_list_2 - np.mean(x_pos_error_list_2))/np.std(x_pos_error_list_2)
# y_pos_error_list_2 = (y_pos_error_list_2 - np.mean(y_pos_error_list_2))/np.std(y_pos_error_list_2)
# print(x_pos_error_list_2)
# # print("Angle error list 2: ",angle_error_list_2.shape)
# print("X position error list 2: ",x_pos_error_list_2.shape)
# print("Y position error list 2: ",y_pos_error_list_2.shape)
x_coord_error_array = np.array(x_pos_error_list_2)
y_coord_error_array = np.array(y_pos_error_list_2)
angle1_error_array = np.array(angle1_error_list_2)
angle2_error_array = np.array(angle2_error_list_2)


Merge them

In [ ]:
# # angle_error_array = np.concatenate((angle_error_list_1,angle_error_list_2),axis=0)
# x_coord_error_array = np.concatenate((x_pos_error_list_1,x_pos_error_list_2),axis=0)
# y_coord_error_array = np.concatenate((y_pos_error_list_1,y_pos_error_list_2),axis=0)
# print("Angle error array: ",angle_error_array.shape)
# print("X position error array: ",x_coord_error_array.shape)
# print("Y position error array: ",y_coord_error_array.shape)

In [ ]:
# #merge both angle_error_list_1 and angle_error_list_2 to a single list and make it a numpy array called 'angle_error_array'
# # angle_error_array = np.array(angle_error_list_1 + angle_error_list_2 )
# angle_error_array = angle_error_list_2
# #merge both x_pos_error_list_1 and x_pos_error_list_2 to a single list and make it a numpy array called 'x_coord_error_array'
# # x_coord_error_array = np.array(x_pos_error_list_1 + x_pos_error_list_2)
# x_coord_error_array = x_pos_error_list_2
# #merge both y_pos_error_list_1 and y_pos_error_list_2 to a single list and make it a numpy array called 'y_coord_error_array'
# # y_coord_error_array = np.array(y_pos_error_list_1 + y_pos_error_list_2)
# y_coord_error_array = y_pos_error_list_2

In [ ]:
# data = np.array([angle_error_array,x_coord_error_array,y_coord_error_array])
# target = np.array([angle_error_array,x_coord_error_array,y_coord_error_array])

In [ ]:
# Here we use 80% of the collected data  as the training set and 20% of the collected data as test set.
#TODO: Assign different propotions of the collected data set and test set and check how the test set error varies of the 
#Neural Network
data =  np.vstack((x_coord_error_array,y_coord_error_array)).T
target = np.vstack(angle_error_array)
# print(data)
# print(target)
data_input_tensor = torch.tensor(data.tolist()).float()
data_target_tensor = torch.tensor(target.tolist()).float()
data_with_target = torch.cat((data_input_tensor,data_target_tensor),1)
#TODO: what is the importance of using DataLoader utility function here?
loader= torch.utils.data.DataLoader(data_with_target,
                                     batch_size=data_with_target.size()[0], shuffle=True,
                                     num_workers=0)
#training set
train_set = []
#test set
test_set = []
for i in iter(loader):
    train_set_index = (int)(np.round(i.shape[0]*0.8))
    train_set = i[:train_set_index,:]
    test_set = i[train_set_index:,:]

In [ ]:
print(train_set.shape)
print(test_set.shape)

In [ ]:
#Defining Neural Network Model
class NN(torch.nn.Module):
    def __init__(self,n_feature,n_hidden1,n_hidden2,n_output):
        super(NN,self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature,n_hidden1)
        #self.do1 = torch.nn.Dropout(0.15)
        #self.relu1 = torch.nn.LeakyReLU()
        #self.bn1 = torch.nn.BatchNorm1d(n_hidden1,affine=False)
        self.hidden2 = torch.nn.Linear(n_hidden1,n_hidden2)
        #self.bn2 = torch.nn.BatchNorm1d(n_hidden2,affine=False)
        #self.relu2 = torch.nn.LeakyReLU()
        #self.do2 = torch.nn.Dropout(0.1)
        self.predict = torch.nn.Linear(n_hidden2,n_output)
        
        
    def forward(self,x):
        x = self.hidden1(x)
        x = torch.sigmoid(x)
        #x = self.do1(x)
        x = self.hidden2(x)
        x = torch.sigmoid(x)
        #x = self.do2(x)
        x = self.predict(x)
        return x

In [ ]:
#instantiate the Neural Network
#model = NN(n_feature=2,n_hidden1=17,n_hidden2=7, n_output=1)
model = NN(n_feature=2,n_hidden1=100,n_hidden2=100, n_output=1)

In [ ]:
#Define loss function : 
# here we use Mean Square Error as the loss function
loss_func = torch.nn.MSELoss()

In [ ]:
#Define the optimizer that should be used in training the Neural Network.
# Here 'lr' is the learning rate
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
train_set_inputs = train_set[:,:2]
#TODO: calculate the mean value of the train_set_inputs. 
mean_of_train_input = torch.mean(train_set_inputs,0)
#standard deviation of the train set inputs.
std_of_the_train_input = torch.std(train_set_inputs,0)
#here we normalize the inputs of the neural network. What is the importance of that?
normalized_train_set_inputs = (train_set_inputs - mean_of_train_input)/std_of_the_train_input

In [ ]:
#targets of the training set
train_set_targets = train_set[:,2][:,np.newaxis]
# train_set_inputs = train_set[:,:2]

In [ ]:
print(normalized_train_set_inputs.shape)
print(train_set_targets.shape)
# print(train_set_inputs)

In [ ]:
#TODO: train the Neural network model by changing the hyper parameters such as learning rate, number of epochs, number of neurons in hidden layers of the neural network.
# What is the minimum mean square error that you can achieve as your neural network converges for the training set.
#  (you will be able to achive a MSE of less than 10 as the Neural network converges.)
num_epochs = 10000
losslist = []
for _ in range(num_epochs):
    prediction = model(normalized_train_set_inputs) # Forward pass prediction. Saves intermediary values required for backwards pass
    loss = loss_func(prediction, train_set_targets) # Computes the loss for each example, using the loss function defined above
    optimizer.zero_grad() # Clears gradients from previous iteration
    loss.backward() # Backpropagation of errors through the network
    optimizer.step() # Updating weights
    # print("prediction =",prediction)
    # print("Loss: ", loss.detach().numpy())
    losslist.append(loss.detach().numpy())

In [ ]:
#plot the mean square error in each epoch/iteration
plt.plot(np.arange(len(losslist)),losslist)
plt.show()

In [ ]:
#save the best neural network model you have obtained.
torch.save(model.state_dict(), 'best_nn_model.pth')

#Save the mean and standard deviation of the train set inputs because we need to use them at test time.

In [ ]:
#reload the your best neural network model with saved parameters
model = NN(n_feature=2,n_hidden1=100,n_hidden2=100, n_output=1)
model.load_state_dict(torch.load('best_nn_model.pth'))

In [ ]:
#TODO: Extract inputs of the test_set
test_set_inputs = test_set[:,:2]
#TODO: Extract test set targets from the test_set
test_set_targets = test_set[:,2][:,np.newaxis]
print(test_set_inputs)

In [ ]:
#TODO: Normalize test set inputs by using the mean and standard deviation of the inputs of the training set
# train_numpy = train_set_inputs.numpy()
# mean_of_train_input = np.mean(train_numpy, axis = 0)
# print(test_numpy)
# print(mean_of_test_input)
# std_of_the_train_input = np.std(train_numpy,0)
normalized_test_set_inputs = (test_set_inputs - mean_of_train_input)/std_of_the_train_input
print(normalized_test_set_inputs)

In [ ]:
#TODO: feed the normalized test set inputs to the Neural Network model and obtain the prediction for the test set.
prediction_test = model(normalized_test_set_inputs)

In [ ]:
print(prediction_test.shape)

In [ ]:
#plot the prediction error of the test set
test_set_prediction_error = prediction_test - test_set_targets
print(prediction_test)
# print(test_set_prediction_error)
plt.plot(np.arange(len(test_set_prediction_error.tolist())),test_set_prediction_error.tolist())

In [ ]:
# In the example model trained with about 600 data, from a test set of 165 samples,
# 159 samples are predicted with prediction error less than 10.

In [ ]:
#TODO: Based on the prediction error of the test set, you can try to train the neural network again by changing the hyper parameters mentioned above.
# Also Try to add Dropout layers to the Neural network and check whether test prediction errors can be reduced further.

# DETECTING THE OBJECT AT TESTING PHASE

In [ ]:
#Here we implement the control loop which is having Neural Network as the controller.
#In this case we donot integrate CMAC to the control loop
def ControlLoopWithNNWithoutCMAC(target__x_coordinate,target__y_coordinate):
    
    number_of_iterations_for_convergence = 0
    #TODO:Intialize your best neural network model and load the saved paramemeters
    NN_model = NN(n_feature=2,n_hidden1=100,n_hidden2=100, n_output=1)
    NN_model.load_state_dict(torch.load('best_nn_model.pth'))

    #TODO: Obtain the x and y coodinates of the green box placed on the end effector of the robot
    x_prev = 0
    y_prev = 0
    
    #Here we loop for 50 iterations assuming that 
    # the controller should achieve the desired target within atmost 50 iterations
    for i in range(1000):
        cam = ct.prepare_camera()
        img = ct.capture_image(cam)
        x,y = ct.locate(img,low_green, high_green)
        cam_red = ct.prepare_camera()
        img_red = ct.capture_image(cam)
        x_red,y_red = ct.locate(img,low_red, high_red)
        print(x,y)
        robot_x_coord_in_image = x
        robot_y_coord_in_image = y
        x_coord_error = x_red - robot_x_coord_in_image;
        y_coord_error = y_red - robot_y_coord_in_image;
        # print(x_coord_error,y_coord_error)
        # x_coord_error = (x_coord_error-mean_x_pos_2)/std_x_pos_2
        # y_coord_error = (y_coord_error-mean_y_pos_2)/std_y_pos_2
        x_prev = x_coord_error
        y_prev = y_coord_error
        print(x_coord_error,y_coord_error)
        #Here if the errors are less than twenty pixels we assume robot reaches the target. 
        # However you can choose any reasonable threshold value instead of 20.
        # if (np.abs(x_coord_error) < (1+x_prev*1.02) and np.abs(y_coord_error) < (1+y_coord_error*1.02)):
        #     print("number of iterations for convergence = ", number_of_iterations_for_convergence)
        #     break
        xy_input_nn_model = [x_coord_error,y_coord_error]
        # print(xy_input_nn_model)
        xy_input_nn_model = np.array(xy_input_nn_model)
        #TODO: normalize the input to the Neural network model using meaning and variance of the training set inputs.
        normalize_xy_input_nn_model = (torch.tensor(xy_input_nn_model) - mean_of_train_input)/std_of_the_train_input
        # normalize_xy_input_nn_model = xy_input_nn_model
        # print(normalize_xy_input_nn_model)
        # normalize_xy_input_nn_model_tensor = torch.tensor(normalize_xy_input_nn_model.tolist()).float()
        # print(normalize_xy_input_nn_model_tensor)
        prediction_for_Y_pos_increment = NN_model(normalize_xy_input_nn_model)
        print(prediction_for_Y_pos_increment)
        #TODO: Get the current Y (angular) position/angle of the robot. you can use the api.getPos function of fable
        robot_current_Y_pos = api.getPos(1,moduleID)
        #Next Y angular position of the robot will be robot_Y_pos + prediction_for_Y_pos_increment
        robot_next_Y_pos = robot_current_Y_pos + prediction_for_Y_pos_increment
        # print(robot_next_Y_pos)
        #TODO: Set the next position of the robot to (-90,robot_next_Y_pos) using the setPos function of the fable.
        api.setPos(-90,robot_next_Y_pos,moduleID)
        api.sleep(1.5)
        number_of_iterations_for_convergence = number_of_iterations_for_convergence + 1
    return


In [ ]:
#TODO: Detect the target object and obtain the coordinates of the object in the image
cam = ct.prepare_camera()
img = ct.capture_image(cam)
# ct.show_camera(cam, low_green, high_green)
x,y = ct.locate(img, low_green, high_green)
print(x,y)
target_x = x
target_y = y
print("target_x = ",target_x)
print("target_y = ",target_y)

In [ ]:
api.setPos(-90,-90,moduleID)

In [ ]:
#TODO: Call the control loop for a target which is detected. Record the number of iterations that the control loop spent for convergence.
ControlLoopWithNNWithoutCMAC(target_x,target_y)
#TODO: change your target location and try again. You may change the target 4-5 times and check how the control loop work.
#Record the number of iterations that the control loop spent for convergence.

In [ ]:
#Now integrate the CMAC to the previous control loop which had only the Neural Network. 
#The implementation of the CMAC can be found in code given for second week exercises.
#TODO: Implement the control loop with both neural network and CMAC. 
def ControlLoopWithBothNNandCMAC(target_x, target_y, iter_num):

    target_x_coord = target_x
    target_y_coord = target_y
    number_of_iterations_for_convergence = 0
    max_iter = iter_num
    #TODO:Intialize your best neural network model and load the saved paramemeters
    NN_model = NN(n_feature=2,n_hidden1=100,n_hidden2=100, n_output=1)
    NN_model.load_state_dict(torch.load('best_nn_model.pth'))

    ## TODO: CMAC initialization
    n_rfs = 20
    beta = 10

    xmin = [-1,-1]
    xmax = [1, 1]

    c = CMAC(n_rfs, xmin, xmax, beta)

    ## Simulation loop
    for i in range(max_iter):
       
        cam = ct.prepare_camera()
        img = ct.capture_image(cam)
        x,y = ct.locate(img,low_green, high_green)
        cam_red = ct.prepare_camera()
        img_red = ct.capture_image(cam)
        x_red,y_red = ct.locate(img,low_red, high_red)
        # print(x,y)
        robot_x_coord_in_image = x
        robot_y_coord_in_image = y
        x_coord_error = x_red - robot_x_coord_in_image;
        y_coord_error = y_red - robot_y_coord_in_image;
        # print(x_coord_error,y_coord_error)
        # x_coord_error = (x_coord_error-mean_x_pos_2)/std_x_pos_2
        # y_coord_error = (y_coord_error-mean_y_pos_2)/std_y_pos_2
        x_prev = x_coord_error
        y_prev = y_coord_error
        
        ## TODO: Implement the CMAC controller into the loop
        
        x_cmac = [x_red ,robot_x_coord_in_image]
        angle_cmac = c.predict(x_cmac)

        # Iterate simulation dynamics
        
    
        #Here if the errors are less than twenty pixels we assume robot reaches the target. 
        # However you can choose any reasonable threshold value instead of 20.
        # if (np.abs(x_coord_error) < (1+x_prev*1.02) and np.abs(y_coord_error) < (1+y_coord_error*1.02)):
        #     print("number of iterations for convergence = ", number_of_iterations_for_convergence)
        #     break
        
        xy_input_nn_model = [x_coord_error,y_coord_error]
        # print(xy_input_nn_model)
        xy_input_nn_model = np.array(xy_input_nn_model)
        #TODO: normalize the input to the Neural network model using meaning and variance of the training set inputs.
        normalize_xy_input_nn_model = (torch.tensor(xy_input_nn_model) - mean_of_train_input)/std_of_the_train_input
        # normalize_xy_input_nn_model = xy_input_nn_model
        # print(normalize_xy_input_nn_model)
        # normalize_xy_input_nn_model_tensor = torch.tensor(normalize_xy_input_nn_model.tolist()).float()
        # print(normalize_xy_input_nn_model_tensor)
        prediction_for_Y_pos_increment = NN_model(normalize_xy_input_nn_model)
        # print(prediction_for_Y_pos_increment)
        #TODO: Get the current Y (angular) position/angle of the robot. you can use the api.getPos function of fable
        robot_current_Y_pos = api.getPos(1,moduleID)
        #Next Y angular position of the robot will be robot_Y_pos + prediction_for_Y_pos_increment
        robot_next_Y_pos = robot_current_Y_pos + prediction_for_Y_pos_increment
        # print(robot_next_Y_pos)
        # print(prediction_for_Y_pos_increment.detach().numpy())
        c.learn(prediction_for_Y_pos_increment.detach().numpy())
        print(angle_cmac)
        robot_nn_and_cmac = robot_next_Y_pos + angle_cmac

        #TODO: Set the next position of the robot to (-90,robot_next_Y_pos) using the setPos function of the fable.
        api.setPos(-90,robot_nn_and_cmac,moduleID)
        api.sleep(1.5)
        number_of_iterations_for_convergence = number_of_iterations_for_convergence + 1
        

    return

In [ ]:
ControlLoopWithBothNNandCMAC(target_x,target_y, 1000)

In [ ]:
#TODO: Compare the number of iteration it takes for convergence in the control loop with 
# neural network only and with both CMAC and neural network.